In [5]:
import pandas as pd
import datetime
from pmdarima import auto_arima
import seaborn as sns
import numpy as np
import math

from fuzzywuzzy import fuzz
from fuzzywuzzy import process


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, \
                            mean_squared_error, median_absolute_error


In [27]:
## Lendo DF
df_model = pd.read_csv('../datasets/df_tratado_final.csv', sep=';')

## Selecionando colunas que serão utilizadas
df_model = df_model[['Produto',
                    'UF',
                    'Semana',
                    'Mes',
                    'Ano',
                    #'preço2',
                    #'Mg %',
                    #'Margem Líquida Real %',
                    'Preço',
                    'Vendas Reais Qtde',
                    #'Vendas Reais $',
                    #'Margem Líquida Real $',
                    #'Venda Líquida Real $',
                    'CMV',
                    #'CMV/QTD',
                    'Região',
                    #'DPPsic',
                    #'Correlacao',
                    #'NObs'
                    ]]

## Selecionando apenas registros onde a observação é maior do que 16
#df_model = df_model.loc[df_model['NObs'] >= 16]

## Removendo o número de observações, porque só foi utilizado para filtrar os registros
#df_model.drop('NObs', axis=1, inplace = True)

## Concatenando as colunas de Ano e Semana
df_model['Semana_Ano'] = df_model['Ano'].map(str) + '-' + df_model['Semana'].map(str)

## Função lambda para percorrer a coluna 'Semana_Ano' aplicar na coluna 'PDia_Semana' o valor do primeiro dia da semana
df_model['PDia_Semana'] = df_model['Semana_Ano'].apply(lambda x: datetime.datetime.strptime(x + '-1', "%Y-%W-%w"))

## Selecionando a coluna de data(PDia_Semana) como index
#df_model.set_index('PDia_Semana', inplace = True)

## Dropando colunas que não serão mais utilizadas
df_model.drop(columns=['Semana_Ano'], axis=1, inplace=True)

In [64]:
## Lendo DF
df_model1 = df_model.copy()

## Selecionando colunas que serão utilizadas
df_produto = df_model1['Produto'].unique().copy()
df_produto = df_produto.tolist()

df_produto = list(filter(lambda x: str(x)!="nan", df_produto))

df_model1['GrpProd']= ""
df_model1['GrpProd'] = df_model1['Produto'].apply(lambda x: 
                                                  
                                                    'AMACIANTE' if 
                                                    (x == 'AMACIANTE COMFORT CONCENTRADO 500ML') |
                                                    (x == 'AMACIANTE COMFORT PURO CUIDADO 500ML' ) |
                                                    (x == 'AMACIANTE DOWNY LIRIOS DO CAMPO 500ML')

                                                    else 'CIF' if 
                                                    (x == 'CIF ULT RAP TIRA LIMO C/CLORO SQZ 500ML' ) |
                                                    (x == 'CIF ULTRA RAP BANH S/CLORO SQUEEZE 500ML')

                                                    else 'DESINFETANTE' if 
                                                    (x == 'DESINF LIQ PINHO SOL CITRUS LIMAO 500ML') |
                                                    (x == 'DESINF LIQ PINHO SOL ORIGINAL 500ML')

                                                    else 'FRONHA' if 
                                                    (x == 'FRONHA AVULSA ESTAMPADA ANDREZA') |
                                                    (x == 'FRONHA AVULSA ESTAMPADA CAMESA' ) |
                                                    (x == 'FRONHA AVULSA LISA ANDREZA') |
                                                    (x == 'FRONHA AVULSA LISA CAMESA')

                                                    else 'RP_CAMA' if 
                                                    (x == 'JG CAMA QUEEN MICROFIBRA ANDREZA') |
                                                    (x == 'JG CAMA QUEEN PRATA 150FIOS MALIBU SANTI')

                                                    else 'CABIDE' if 
                                                    (x == 'KIT CABIDE C/3 MADEIRA REV')

                                                    else 'PANO_LIMP' if 
                                                    (x == 'PANO SCOTT DURAMAX LIMPEZA DIARIA C/5')

                                                    else 'TOALHA')


In [33]:
df_model1

,Produto,UF,Semana,Mes,Ano,Preço,Vendas Reais Qtde,CMV,Região,PDia_Semana,GrpProd
0,T VISITA P BORDA DESIRE AZUL 6238,AL,1,1,2017,4.74000,7.0,12.44,Nordeste,2017-01-02,TOALHA
1,T VISITA P BORDA DESIRE AGUA 6243,AL,1,1,2017,4.99000,7.0,11.97,Nordeste,2017-01-02,TOALHA
2,AMACIANTE COMFORT CONCENTRADO 500ML,AL,1,1,2017,7.96619,99.0,427.30,Nordeste,2017-01-02,AMACIANTE
3,T VISITA P BORDA DESIRE AZUL 6238,AM,1,1,2017,4.74000,4.0,6.36,Norte,2017-01-02,TOALHA
4,T VISITA P BORDA DESIRE AGUA 6243,AM,1,1,2017,4.99000,15.0,23.30,Norte,2017-01-02,TOALHA
...,...,...,...,...,...,...,...,...,...,...,...
9415,T VISITA DISNEY CARS SANTISTA,TO,52,12,2017,4.99000,3.0,5.92,Norte,2017-12-25,TOALHA
9416,PANO SCOTT DURAMAX LIMPEZA DIARIA C/5,TO,52,12,2017,7.99000,4.0,11.25,Norte,2017-12-25,PANO_LIMP
9417,NaN,TO,52,12,2017,4.99000,14.0,24.58,Norte,2017-12-25,TOALHA
9418,CIF ULTRA RAP BANH S/CLORO SQUEEZE 500ML,TO,52,12,2017,3.95000,14.0,33.04,Norte,2017-12-25,CIF


In [34]:
## Criando um cópia do df_model
df_model2 = df_model1.copy() 

In [35]:
## Calculando preço x quantidade
df_model2['PrecoXQtd'] = df_model2['Preço'] * df_model2['Vendas Reais Qtde'] 

In [37]:
df_agg = df_model2[["UF", "GrpProd", "Semana", "Vendas Reais Qtde", 'PrecoXQtd', 'CMV']]\
                    .groupby(["UF", "GrpProd", "Semana"]).sum().reset_index()

 
#df_agg_sum = df[["SAP+NOME", "UF", "Ano", "Semana", "Vendas Reais Qtde", "Vendas Reais $",
#                     "Margem Líquida Real $", "Venda Líquida Real $", "CMV"]]. \
#             groupby(["SAP+NOME", "UF", "Semana", "Ano"]).sum().reset_index()
#df_agg_sum["CMV/QTD"] = df_agg_sum["CMV"] / df_agg_sum["Vendas Reais Qtde"]

In [39]:
df_agg

,UF,GrpProd,Semana,Vendas Reais Qtde,PrecoXQtd,CMV
0,AC,AMACIANTE,3,10.0,79.90,43.84
1,AC,AMACIANTE,13,4.0,19.96,17.28
2,AC,AMACIANTE,14,17.0,84.83,73.41
3,AC,AMACIANTE,24,3.0,29.97,19.11
4,AC,AMACIANTE,25,4.0,31.96,26.32
...,...,...,...,...,...,...
4973,TO,TOALHA,48,12.0,61.88,23.70
4974,TO,TOALHA,49,33.0,148.67,65.80
4975,TO,TOALHA,50,41.0,182.59,78.40
4976,TO,TOALHA,51,38.0,165.62,74.48


In [40]:
df_agg['PrMedPonderado'] = df_agg['PrecoXQtd']  / df_agg['Vendas Reais Qtde']

In [58]:
df_num_obs = df_agg.groupby(['UF', 'GrpProd']).Semana.nunique().reset_index()

In [59]:
df_model_final = pd.merge(df_agg, df_num_obs, how='left', on=['UF', 'GrpProd'])

df_model_final.rename(columns={"Semana_y":'NObs', "Semana_x":'Semana'}, inplace = True)

In [61]:
df_model_final = df_model_final.sort_values(by='NObs', ascending=False)

In [63]:
df_model_final.to_csv('../datasets/df_model_final.csv', index=False)